## Step1 資料庫連線

In [50]:
#把使用的套件都匯入
import re
from pymongo import MongoClient 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import jieba
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("E:/dics/dict_keyw.txt")       #加入自建詞庫

#預設就是自己
client = MongoClient('10.120.28.12',27017)
database = client['test']
collection =database['test']

Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.572 seconds.
Prefix dict has been built succesfully.


## Step2 獲取資料

In [51]:
import time
tic = time.clock()

title=[]
content =[]
date=[]
#拿資料
for post in collection.find(): 
    summary = post['content']
    content.append('/'.join(jieba.cut(summary)))
    title.append(post['title'])
    date.append(post['date'])
#總文章數量
print "文章數:",len(title)
toc = time.clock()
print "執行時間:",(toc - tic)/60

文章數: 7135
執行時間: 0.24913196658


## Step3 計算TF-IDF

In [52]:
#定義計算TF-IDF的函式
def tfIdf(content):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(content)  # titile 放文本
    weight = X.toarray()
    print "特徵值數量:",len(vectorizer.get_feature_names())
    return weight,X,vectorizer

#把上面存好的切詞進行TF-IDF計算，並把結果存進 weight
weight,X,vectorizer = tfIdf(content)

特徵值數量: 123652


## Step4 匯入計算函式 (擇一個版本執行)

In [13]:
#正常版
def getTopTfidf(news_id,n):
    indices = np.argsort(weight[news_id])[::-1]  
    features = vectorizer.get_feature_names()     
    print title[news_id]
    top_n = n
    top_features = [features[i] for i in indices[:top_n]]
    return top_features,indices


def getTopWeight(news_id,n):
    top_features,indices = getTopTfidf(news_id,n)
    a=0
    for i in top_features:
        print i,weight[news_id][indices[a]]  # weight[2000] 裡面請填入跟上面一樣的文章編號
        a=a+1

In [53]:
#class 版
#方法內的self 指的是 class
#當該class有被使用時，會像參數一樣被入方法內(self名子可以自己取)

class tfidf():
    
    #@staticmethod #宣告成靜態方法, 靜態方法不需要設定self
    def getTopTfidf(self,news_id,n):
        indices = np.argsort(weight[news_id])[::-1]  
        features = vectorizer.get_feature_names()     
        #print title[news_id]
        #print date[news_id]
        top_n = n
        top_features = [features[i] for i in indices[:top_n]]
        return top_features,indices

    #@staticmethod #宣告成靜態方法, 靜態方法不需要設定self
    def getTopWeight(self,news_id,n):
        top_features,indices = tfidf().getTopTfidf(news_id,n)  #類別名稱.方法()
        tfidfword=""
        a=0
        for i in top_features:
            #print i,weight[news_id][indices[a]]  # weight[2000] 裡面請填入跟上面一樣的文章編號
            a=a+1
            tfidfword += i+" "  #把關鍵字都存起來
        return tfidfword
#def getTopWeight(self,news_id,n):  #這是定義實體/非靜態方法, self指的是"類別物件()"本身
    #呼叫實體/非靜態方法, 類別物件().方法()
    #沒有做任何宣告的方法預設會是實體/非靜態方法
    
    #python 還有定義一個類別方法 @classmethod

## Step5 印出某篇文章TOP 特徵詞

In [61]:
##請在這邊輸入要搜尋的文章
#第一個參數輸入文章編號、第二個參數輸入TOP多少的詞        

#一般版本
#getTopWeight(30,50) 

#呼叫class版本
tfidf().getTopWeight(964,50)    #test().getTopWeight(30,50) #呼叫實體/非靜態方法, 類別物件().方法() 

#把關鍵字存起來
tfidfWord = tfidf().getTopWeight(964,50)

## Step6 把找到的關鍵字存入資料庫中

In [77]:
obj = collection.find()
print type(obj)
print tfidfWord
print obj[1]["title"]
print type(tfidfWord)
print type(obj[1]["title"])

<class 'pymongo.cursor.Cursor'>
警大 回鄉 考上 大弟 姊夫 警員 阿公 阿嬤 念書 年關 寶貝 之母 上進心 表率 兩個 警界 妹婿 情怯 妻小 有樣學樣 勤奮 父母親 我們 爸爸 雙親 有加 小孩 搶著 立即 過年 
燦爛跨年,歡動全台
<type 'unicode'>
<type 'unicode'>


In [80]:
import time
tic = time.clock()

tfidfWord = []
for i in range(0,7134):
    tfidfWord.append(tfidf().getTopWeight(i,30))
    
toc = time.clock()
print "執行時間:",(toc - tic)/60

KeyboardInterrupt: 

In [76]:
import time
tic = time.clock()

obj = collection.find()  #這行一定要加
for i in range(0,7134):
    database.collection.update({'title':obj[i]['title']}, {'tfidf': tfidfWord[i]})
    
toc = time.clock()
print "執行時間:",(toc - tic)/60

KeyboardInterrupt: 